In [1]:
!pip install langchain qdrant_client openai tiktoken
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
import qdrant_client
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings


In [7]:
os.environ['qdrant_host']="your_link"
os.environ['qdrant_api_key']="Your_key"


client=qdrant_client.QdrantClient(
    os.getenv("qdrant_host"),
    api_key=os.getenv("qdrant_api_key")
    )

In [8]:
os.environ['qdrant_collection_name']='shiya-collection'
#open ai embeddings used which have a size of #1536
client.recreate_collection(
    collection_name=os.getenv("qdrant_collection_name"),
    vectors_config=qdrant_client.http.models.VectorParams(size=1536, distance=qdrant_client.http.models.Distance.COSINE),
)

<ipython-input-8-95cbb08b9175>:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [ ]:
os.environ["OpenAI_API_KEY"]="your_api_key"
embedding=OpenAIEmbeddings()
vector_store=Qdrant(
    client=client,
    collection_name=os.getenv("qdrant_collection_name"),
    embeddings=embeddings,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
def get_chunks(text, chunk_size=1000, chunk_overlap=20):
  text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap
                                        separator="\n")
  chunks = text_splitter.split_text(text)
  return chunks
file_paths = [
    '/content/shoedocuments/NikeAirforce.txt',
    '/content/shoedocuments/Nikeahistory.txt',
    '/content/shoedocuments/jordanswiki.txt'
]
for file_path in file_paths:
    with open(file_path) as file:
      raw_text=file.read()
    texts=get_chunks(raw_text)
    vector_store.add_texts(texts)



In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
qa=RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)